# CDSAPI request examples  
## Python workshop, EMS2019

### A few tips before we begin

- Use CDS Web download to construct the base of your request and then build on it. 
- Reanalysis ERA5 data is originally stored in GRIB format and when you download it as netCDF, conversion will fail if there is more than one **Product type** in the request
- Data that is originally stored as netCDF will be offered to be downloaded as zip or Compressed tar file. If you are downloading just one file (selected exactly one choice in every category) - change the extension to **.nc** (but leave format as zip)
- For people who are used to using MARS archive - for keys that are the same as in MARS request (area, time and date for daily data) you can use mars syntax in some requests

## Examples list 
[Example 1 - Seasonal forecast](#example1)  
[Example 2 - ERA5 Land Total precipitation data](#example2)  
[Example 3 - ERA5 Temperature on pressure levels over Europe](#example3)  
[Example 4 - Hourly ERA5 data in netcdf format](#example4)  
[Example 5 - Hourly ERA5 Land data in grib format for small area](#example5)  
[Example 6 - Satellite observation data](#example6)  
[Example 7 - Satellite observation data - one file in request](#example7)  
[Example 8 - Climate projections data](#example8)  
[Example 9 - ERA5 Monthly data](#example9)  
[Example 10 - ERA5 data on model levels from public datasets in MARS](#example10)  
[Example 11 - UERRA regional reanalysis for Europe on height levels](#example11)  
[Example 12 - ERA5 reanalysis - ensemble data on pressure levels](#example12) 

Before we start, we need to import cds and datetime libraries

In [ ]:
import cdsapi
from datetime import date, timedelta
import xarray as xr

In [ ]:
c = cdsapi.Client()

<a id='example1'></a>
### Example 1 - Seasonal forecast
Get the latest seasonal forecast anomalies on single levels from ECMWF and UK Met Office. We have to use the download form to know which System is available from which model for the period we want the forecast.  

In [ ]:
c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format':'grib',
        'originating_centre':[
            'ecmwf','ukmo'
        ],
        'system':[
            '5','14'
        ],
        'variable':'2m_temperature_anomaly',
        'product_type':'monthly_mean',
        'year':'2019',
        'month':'05',
        'leadtime_month':[
            '1','2'
        ]
    },
    'data/weather/seasonal.grib')

<a id='example2'></a>
### Example 2 - ERA5 Land Total precipitation data
Get the daily total precipitation data from ERA5 Land dataset for month of January 2012.  
In ERA5 Land dataset total precipitation is archived as acumulations from start of the period, which in this case is from start of the day. So at 23h time it will contain daily precipitations.  
ERA5 Land only has high resolution model, so there is no 'product_type' parameter.

In [ ]:
c.retrieve(
    'reanalysis-era5-land',
    {
        'variable':[
            'total_precipitation'
        ],
        'year':'2012',
        'month':'01',
        'day':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12',
            '13','14','15',
            '16','17','18',
            '19','20','21',
            '22','23','24',
            '25','26','27',
            '28','29','30',
            '31'
        ],
        'time': '23:00',
        'format':'netcdf'
    },
    'data/weather/total_precipitation_2012_01.nc')

<a id='example3'></a>
## Example 3 - ERA5 Temperature on pressure levels over Europe
Get temperature at 850 hPa and 500 hPa for days between 15th and 17th August 2008 and times at 00, 06, 12 and 18 UTC. Save the result in separate daily netCDF files.  
Note that instead of giving day, month and year separately, we can just give 'date'.  
Get the data for Europe area. Just add **'area' : 'europe'**. Unfortunatelly this is the only shortcut word that I've found to work 🙁

In [ ]:
sdate = date(2008, 8, 15)   # start date
edate = date(2008, 8, 17)   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    d = sdate + timedelta(days=i)
    d = str(d)

    c.retrieve("reanalysis-era5-pressure-levels",
                       {
                           "variable": "temperature",
                           "pressure_level": ['850','500'],
                           "product_type": "reanalysis",
                           "area" : "europe",
                           "date": d,
                           "time": ['0','6','12','18'],
                           "format": "netcdf"
                       },
                       "data/weather/ea_t_{day}.nc".format(day=d)
                       )

<a id='example4'></a>
### Example 4 - Hourly ERA5 data in netcdf format
Get hourly data from ERA5 single level dataset for dates from 28th April to 3rd May 2018 for global area.  
Get these variables:
- wind gusts
- minimum 2m temperature 
- maximum 2m temperature  

In previous example we had a request for each day. This time we we make one request for all the dates.  
Giving the list of dates is also the only way to get the data for different days in different months.  
Instead of a whole list of numbers from 0 to 23, we can just put **'all'** for time.

In [ ]:
c.retrieve("reanalysis-era5-single-levels",
                       {
                           "variable":["10m_wind_gust_since_previous_post_processing",
                                       "maximum_2m_temperature_since_previous_post_processing",
                                       "minimum_2m_temperature_since_previous_post_processing"],
                           "product_type": "reanalysis",
                           "date": ['20180428',
                                    '20180429',
                                    '20180430',
                                    '20180501',
                                    '20180502',
                                    '20180503'],
                           "time": ['all'],
                           "grid": [1,1],
                           "format": "grib"
                       },
                       "data/weather/min_max_t_and_wind_gust.grib"
                       )

<a id='example5'></a>
### Example 5 - Hourly ERA5 Land data in grib format for small area
Get hourly 2m temperature and u and v 10m wind component data for the whole year for years from 2010 to 2013. Each year is one request and saved in one grib file.  
Using **'area'** keyword, get the data only for London area.  
Don’t worry that some combinations don't exist (e.g., April 31), the API will ignore these fields.  

💡 Be careful when using ERA5 Land data. It is very new dataset and it is structured differently than the rest of the ERA5.

Note: **Even thogh the retrived data is very small because we want very small area, the requests themselves are very big, having a year of hourly data with 3 parameters. So it will run for at least 20 minutes. Don't run it at the workshop. Try it out at home.**

In [ ]:
years = ['2010','2011','2012','2013']

for year in years:
    file_name = 'data/weather/london_data_{y}.grib'.format{y=year}
    if use_cds:
        c = cdsapi.Client()
        c.retrieve(    'reanalysis-era5-land',
        {
            'variable':[
                '10m_u_component_of_wind',
                '10m_v_component_of_wind',
                '2m_temperature',
            ],
            'year':[
                year         
            ],
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'
            ],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format':'grib',
            'area':[51,-1,52,1]
                   },
                   file_name)

<a id='example6'></a>
## Example 6 - Satellite data - Sea level anomaly

Get the Sea level gridded data for the global ocean for three days in March 2018.

In [ ]:
c.retrieve(
    'satellite-sea-level-global',
    {
        'variable':'all',
        'format':'zip',
        'year':'2018',
        'month':'03',
        'day':['03', '04', '05']
    },
    'data/weather/satellite.zip')

<a id='example7'></a>
## Example 7 - Satellite data - Soil moisture - one netCDF file

Get the Volumetric surface soil moisture day average data for 2nd June 2019.  
Requests that are originally in netCDF format are available compressed into one zip file.  

💡 When thre is only one netCDF file in zip archive thre will be an error. Before running such requests, change extension in file name from .zip to nc.

In [ ]:
c.retrieve(
    'satellite-soil-moisture',
    {
        'format':'zip',
        'variable':'volumetric_surface_soil_moisture',
        'type_of_sensor':'combined_passive_and_active',
        'time_aggregation':'day_average',
        'year':'2019',
        'month':'06',
        'day':'02',
        'type_of_record':'icdr',
        'version':'v201706.0.0'
    },
    'data/weather/satellite_one.nc')

<a id='example8'></a>
## Example 8 - Climate projections data - 2m temperature and 2m Tmax

Get 2m temperature and Maximum 2m temperature for last 24 hours from GFDL-CM3 (NOAA, USA) model, 
RCP 8.5 experiment and ensemble member r1i1p1 for 2020-2025 period.  

💡 Always use Web download to check which combination of model, experiment and ensamble member is available for the period you're interested.

In [ ]:
c.retrieve(
    'projections-cmip5-monthly-single-levels',
    {
        'ensemble_member':'r1i1p1',
        'format':'zip',
        'variable':[
            '2m_temperature','maximum_2m_temperature_in_the_last_24_hours'
        ],
        'model':'gfdl_cm3',
        'experiment':'rcp_8_5',
        'period':'202101-202512'
    },
    'data/weather/climate_projection.zip')

<a id='example9'></a>
## Example 9 - ERA5 Reanalysis - Monthly data

Get Monthly averaged 2m temperature for first 3 months of 2018 for a small area in grib format.

In [ ]:
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format':'grib',
        'product_type':'monthly_averaged_reanalysis',
        'variable':'2m_temperature',
        'area' : [42,19,17,24],
        'year':'2018',
        'month':['1','2','3'],
        'time':'00:00'
    },
    'data/weather/ERA5_monthly.grib')

<a id='example10'></a>
### Example 10 - ERA5 data on model levels from public datasets in MARS
Get the data on Model levels from ERA5 complete data set.   
Keep in mind that this data is not stored in CDS, but in public part of MARS archive with the access through CDS. Because of this you can expect queuing to last hours or even days.  
💡 If the data you're interested contains **accumulated fields**, which are 'type'=FC you will need to provide **step** too.   
💡 When retrieving data from **reanalysis-era5-complete** dataset **always make sure you have grid parameter** in your request, otherwise the retrieved data will be in default reduced Gaussian grid GRIB file.  

**Don't run this example at the workshop. Try it out at home.**

In [ ]:
c.retrieve('reanalysis-era5-complete',{
    'class':'ea',
    'date':'20180525/to/20180605',
    'area':'50/5/40/27',
    'expver':'1',
    'levelist': '1/to/137',
    'levtype':'ml',
    'param':'129/130/131/132/133/152',
    'stream':'oper',
    'time':'00:00:00/03:00:00/06:00:00/09:00:00/12:00:00/15:00:00/18:00:00/21:00:00',
    'type':'an',
    'grid':"0.25/0.25",
},'data/weather/ERA5-ml.grb')

<a id='example11'></a>
### Example 11 - UERRA regional reanalysis for Europe on height levels
Get the wind data from UERRA regional renalysis on height levels.  
Web download form won't let you select more than one parameter for this dataset, but putting more than one parameter in the API request works.
  
  
Regional reanalysis for European domain is available from 2 models: UERRA-HARMONIE and MESCAN-SURFEX systems. While sufrace data and data on pressure levels ara available from both models, data on height levels in meters is only available from UERRA-HARMONIE. That is why in this dataset there is no 'origin' parameter as in other two UERRA datasets.

In [ ]:
c.retrieve(
    'reanalysis-uerra-europe-height-levels',
    {
        'variable':['wind_direction','relative_humidity'],
        'height_level':[
            '100_m','150_m','15_m',
            '200_m','250_m','300_m',
            '30_m','400_m','500_m',
            '50_m','75_m'
        ],
        'year':'2018',
        'month':'03',
        'day':'03',
        'time':[
            '00:00','06:00','12:00',
            '18:00'
        ],
        'format':'grib'
    },
    'data/weather/UERRA.grib')

<a id='example12'></a>
### Example 12 - ERA5 reanalysis - ensemble data on pressure levels
Download temperature and relative humidity data on levels 300 and 500 hPa for European area for period from 5th to 7th May 2018. Download all ensemble members.

In [ ]:
c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type':'ensemble_members',
        'variable':[
            'relative_humidity','temperature'
        ],
        'pressure_level':[
            '300','500'
        ],
        'year':'2018',
        'month':[
            '05'
        ],
        'day':[
            '05','06','07'
        ],
        'time':[
            '00:00','03:00','06:00',
            '09:00','12:00','15:00',
            '18:00','21:00'
        ],
        'area':[35,-10,52,25],
        'format':'grib'
    },
    'data/weather/ensemble_data_pl.grib')